In [1]:
import pandas as pd
import tensorflow as tf
import glob
import os
import numpy as np
import sklearn.metrics as sk
import tensorflow_datasets as tfds
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import seaborn as sns
import helpers
import dataset
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2022-11-18 16:58:58.144005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 16:58:58.280203: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-18 16:58:58.311573: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-18 16:58:58.811067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries

# MU3D_path = '/Users/frank/Downloads/dataSets/MU3D-dataset/processed/' # 320 entries

In [3]:
X, Y = dataset.path_preprocessing(truth_trial_path, lie_trial_path)

CSV file ./processed_lie/trial_lie_007.csv only has 15 columns


In [4]:
TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [5]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=True)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))

  # model.add(Dropout(drop))
  model.add(Dense(1, activation='linear')) #softmax

  #model.add(Lambda(lambda x: tf.cast(tf.argmax(x), tf.float32)))
  #model.add(Dense(1, activation=activ))

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


2022-11-18 16:59:25.171767: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 16:59:25.202008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 16:59:25.202210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 16:59:25.202986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                6272      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2022-11-18 16:59:28.801227: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


1506/1506 [==============================] - 8s 4ms/step - loss: 0.1038 - accuracy: 0.9501 - val_loss: 0.0438 - val_accuracy: 0.9824
Epoch 2/10
1506/1506 [==============================] - 6s 4ms/step - loss: 0.0203 - accuracy: 0.9931 - val_loss: 0.0084 - val_accuracy: 0.9978
Epoch 3/10
1506/1506 [==============================] - 5s 4ms/step - loss: 0.0120 - accuracy: 0.9962 - val_loss: 0.0057 - val_accuracy: 0.9983
Epoch 4/10
1506/1506 [==============================] - 5s 3ms/step - loss: 0.0081 - accuracy: 0.9974 - val_loss: 0.0027 - val_accuracy: 0.9993
Epoch 5/10
1506/1506 [==============================] - 5s 3ms/step - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.0027 - val_accuracy: 0.9993
Epoch 6/10
1506/1506 [==============================] - 5s 4ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 7.5931e-04 - val_accuracy: 0.9997
Epoch 7/10
1506/1506 [==============================] - 5s 4ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 2.4582e-04 - val_accuracy: 1.